# 9장 - 합성곱 신경망 CNN
---
- 컨볼루션
- 패딩 - 원본 이미지가 줄어드는 것을 방지하기 위해 덧대는 것
- 스트라이드 - 몇 칸을 이동할지, 스트라이드가 `커질수록` 결과가 `작아짐`
- **패딩한 상태**에서 <u>약수</u>를 구해서 스트라이드 할 것
- `(H or W + 2p - FH) / S`

- >풀링 - max pulling이 가장 결과가 좋다고 함
- 바디에서 헤더 넘어갈 때는 항상 비효율적임. 파라미터 잘 고려해야 함

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


* np.expand_dims(변수, axis=-1)

In [10]:
x_train_sc, x_test_sc = x_train / 255, x_test / 255
x_train_sc = np.expand_dims(x_train_sc, axis=-1) # 흑백채널 하나
x_test_sc = np.expand_dims(x_test_sc, axis=-1)
print(x_train_sc.shape)
print(x_test_sc.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


* 1D는 시계열, 3D는 동영상

In [11]:
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D

inputs = Input(shape=(28, 28, 1)) # 흑백채널 하나

x = Conv2D(32, (3, 3), padding='same')(inputs) # 필터 수, 커널사이즈
x = MaxPooling2D((2,2), strides=2)(x)
x = Conv2D(32, (3, 3), padding='same')(x) # 보통 처음 풀링 이후 activation 넣는다.
x = MaxPooling2D((2,2), strides=2)(x)
x = Conv2D(32, (3, 3), padding='same')(x)

x = Flatten()(x) # GlobalAverageFooling2D를 요즘 거의 많이 씀
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 14, 14, 32)        9248      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 7, 7, 32)          9248

In [12]:
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Activation, GlobalMaxPooling2D

inputs = Input(shape=(28, 28, 1)) # 흑백채널 하나

x = Conv2D(128, (3, 3), padding='same')(inputs) # 필터 수, 커널사이즈
x = MaxPooling2D((2,2), strides=2)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(128, (3, 3), padding='same')(x) 
x = MaxPooling2D((2,2), strides=2)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(256, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = GlobalMaxPooling2D()(x) # GlobalAverageFooling2D를 요즘 거의 많이 씀 , 채널을 압축시켜줌
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model2 = keras.Model(inputs=inputs, outputs=outputs)
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 28, 28, 128)       1280      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 14, 14, 128)      0         
 g2D)                                                            
                                                                 
 batch_normalization_6 (Batc  (None, 14, 14, 128)      512       
 hNormalization)                                                 
                                                                 
 activation_6 (Activation)   (None, 14, 14, 128)       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 14, 14, 128)       1475

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

stopper = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model.fit(x_train_sc, y_train, epochs=100, batch_size=128, validation_split=0.2, callbacks=[stopper], verbose=2)

Epoch 1/100
375/375 - 6s - loss: 0.2565 - accuracy: 0.9235 - val_loss: 0.0903 - val_accuracy: 0.9735 - 6s/epoch - 15ms/step
Epoch 2/100
375/375 - 2s - loss: 0.0634 - accuracy: 0.9806 - val_loss: 0.0550 - val_accuracy: 0.9837 - 2s/epoch - 5ms/step
Epoch 3/100
375/375 - 2s - loss: 0.0460 - accuracy: 0.9856 - val_loss: 0.0541 - val_accuracy: 0.9851 - 2s/epoch - 5ms/step
Epoch 4/100
375/375 - 2s - loss: 0.0377 - accuracy: 0.9883 - val_loss: 0.0532 - val_accuracy: 0.9855 - 2s/epoch - 5ms/step
Epoch 5/100
375/375 - 2s - loss: 0.0289 - accuracy: 0.9907 - val_loss: 0.0491 - val_accuracy: 0.9872 - 2s/epoch - 5ms/step
Epoch 6/100
375/375 - 2s - loss: 0.0255 - accuracy: 0.9919 - val_loss: 0.0515 - val_accuracy: 0.9846 - 2s/epoch - 4ms/step
Epoch 7/100
375/375 - 2s - loss: 0.0212 - accuracy: 0.9927 - val_loss: 0.0471 - val_accuracy: 0.9871 - 2s/epoch - 5ms/step
Epoch 8/100
375/375 - 2s - loss: 0.0183 - accuracy: 0.9938 - val_loss: 0.0504 - val_accuracy: 0.9873 - 2s/epoch - 5ms/step
Epoch 9/100
375

- 어제 한 결과보다 더 좋게 나옴

# 모델 저장하는 방법

In [15]:
model.save('models/mnist_cnn.h5')

In [16]:
eval_loss, eval_acc = model.evaluate(x_test_sc, y_test)
print('loss: ', eval_loss, 'acc: ', eval_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.0439 - accuracy: 0.9863
loss:  0.043949194252491 acc:  0.986299991607666
